In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
text = '''Wood, in the strict sense, is yielded by trees, which increase in diameter by the formation, between the existing wood and the inner bark, of new woody layers which envelop the entire stem, living branches, and roots. This process is known as secondary growth; it is the result of cell division in the vascular cambium, a lateral meristem, and subsequent expansion of the new cells. These cells then go on to form thickened secondary cell walls, composed mainly of cellulose, hemicellulose and lignin.

Where the differences between the seasons are distinct, e.g. New Zealand, growth can occur in a discrete annual or seasonal pattern, leading to growth rings; these can usually be most clearly seen on the end of a log, but are also visible on the other surfaces. If the distinctiveness between seasons is annual (as is the case in equatorial regions, e.g. Singapore), these growth rings are referred to as annual rings. Where there is little seasonal difference growth rings are likely to be indistinct or absent. If the bark of the tree has been removed in a particular area, the rings will likely be deformed as the plant overgrows the scar.

If there are differences within a growth ring, then the part of a growth ring nearest the center of the tree, and formed early in the growing season when growth is rapid, is usually composed of wider elements. It is usually lighter in color than that near the outer portion of the ring, and is known as earlywood or springwood. The outer portion formed later in the season is then known as the latewood or summerwood.[9] There are major differences, depending on the kind of wood. If a tree grows all its life in the open and the conditions of soil and site remain unchanged, it will make its most rapid growth in youth, and gradually decline. The annual rings of growth are for many years quite wide, but later they become narrower and narrower. Since each succeeding ring is laid down on the outside of the wood previously formed, it follows that unless a tree materially increases its production of wood from year to year, the rings must necessarily become thinner as the trunk gets wider. As a tree reaches maturity its crown becomes more open and the annual wood production is lessened, thereby reducing still more the width of the growth rings. In the case of forest-grown trees so much depends upon the competition of the trees in their struggle for light and nourishment that periods of rapid and slow growth may alternate. Some trees, such as southern oaks, maintain the same width of ring for hundreds of years. On the whole, as a tree gets larger in diameter the width of the growth rings decreases.'''

import re
text = text.lower()
pattern = r'[^a-zA-z.]'
text = re.sub(pattern, " ", text)
text2 = text.split('.')
text = ''.join(text2)

In [3]:
text = ' '.join(text.split())
text

'wood in the strict sense is yielded by trees which increase in diameter by the formation between the existing wood and the inner bark of new woody layers which envelop the entire stem living branches and roots this process is known as secondary growth it is the result of cell division in the vascular cambium a lateral meristem and subsequent expansion of the new cells these cells then go on to form thickened secondary cell walls composed mainly of cellulose hemicellulose and lignin where the differences between the seasons are distinct eg new zealand growth can occur in a discrete annual or seasonal pattern leading to growth rings these can usually be most clearly seen on the end of a log but are also visible on the other surfaces if the distinctiveness between seasons is annual as is the case in equatorial regions eg singapore these growth rings are referred to as annual rings where there is little seasonal difference growth rings are likely to be indistinct or absent if the bark of 

In [4]:
text3 = []
for i in text2:
    text3.append(' '.join(i.split()))
    
text3 = [i for i in text3 if len(i)>10]
text3

['wood in the strict sense is yielded by trees which increase in diameter by the formation between the existing wood and the inner bark of new woody layers which envelop the entire stem living branches and roots',
 'this process is known as secondary growth it is the result of cell division in the vascular cambium a lateral meristem and subsequent expansion of the new cells',
 'these cells then go on to form thickened secondary cell walls composed mainly of cellulose hemicellulose and lignin',
 'where the differences between the seasons are distinct e',
 'new zealand growth can occur in a discrete annual or seasonal pattern leading to growth rings these can usually be most clearly seen on the end of a log but are also visible on the other surfaces',
 'if the distinctiveness between seasons is annual as is the case in equatorial regions e',
 'singapore these growth rings are referred to as annual rings',
 'where there is little seasonal difference growth rings are likely to be indistinc

In [5]:
tokenizer = Tokenizer()

y = text3

tokenizer.fit_on_texts(y)
mask_token_id = len(tokenizer.word_index)+1
tokenizer.word_index.update({'[mask]': mask_token_id})

In [6]:
def get_masked_input(sequence):
    mask = np.random.randint(low=0, high=len(sequence))
    return [token if i != mask else mask_token_id for i, token in enumerate(sequence)]

y = tokenizer.texts_to_sequences(y)
x = [get_masked_input(seq) for seq in y]

In [7]:
VOCAB = len(tokenizer.word_index)
VOCAB

216

In [8]:
MAX_SEQ_LEN = 50
EMBEDDING_VECTOR_LENGTH = 256

x = pad_sequences(x, maxlen=MAX_SEQ_LEN, padding='post')
y = pad_sequences(y, maxlen=MAX_SEQ_LEN, padding='post')
y = keras.utils.to_categorical(y)

In [9]:
model = keras.models.Sequential([
    L.Embedding(input_dim=VOCAB, output_dim=EMBEDDING_VECTOR_LENGTH, input_length=MAX_SEQ_LEN),
    L.Bidirectional(L.LSTM(512, return_sequences=True)),
    L.TimeDistributed(L.Dense(VOCAB, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 256)           55296     
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 1024)          3149824   
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 216)           221400    
Total params: 3,426,520
Trainable params: 3,426,520
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x=x, y=y,epochs=200)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 5.3616 - accuracy: 0.0010
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 5.1408 - accuracy: 0.5370
Epoch 3/200
1/1 [==============================] - 0s 16ms/step - loss: 4.7839 - accuracy: 0.5370
Epoch 4/200
1/1 [==============================] - 0s 15ms/step - loss: 3.6798 - accuracy: 0.5370
Epoch 5/200
1/1 [==============================] - 0s 15ms/step - loss: 3.5939 - accuracy: 0.5370
Epoch 6/200
1/1 [==============================] - 0s 16ms/step - loss: 3.1232 - accuracy: 0.5370
Epoch 7/200
1/1 [==============================] - 0s 16ms/step - loss: 2.5425 - accuracy: 0.5370
Epoch 8/200
1/1 [==============================] - 0s 16ms/step - loss: 2.6763 - accuracy: 0.5370
Epoch 9/200
1/1 [==============================] - 0s 16ms/step - loss: 2.7263 - accuracy: 0.5390
Epoch 10/200
1/1 [==============================] - 0s 15ms/step - loss: 2.6334 - accuracy: 0.5390
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 15ms/step - loss: 1.6466 - accuracy: 0.6360
Epoch 84/200
1/1 [==============================] - 0s 14ms/step - loss: 1.6228 - accuracy: 0.6310
Epoch 85/200
1/1 [==============================] - 0s 15ms/step - loss: 1.6191 - accuracy: 0.6240
Epoch 86/200
1/1 [==============================] - 0s 15ms/step - loss: 1.6221 - accuracy: 0.6200
Epoch 87/200
1/1 [==============================] - 0s 15ms/step - loss: 1.6063 - accuracy: 0.6320
Epoch 88/200
1/1 [==============================] - 0s 16ms/step - loss: 1.5331 - accuracy: 0.6510
Epoch 89/200
1/1 [==============================] - 0s 14ms/step - loss: 1.5430 - accuracy: 0.6380
Epoch 90/200
1/1 [==============================] - 0s 16ms/step - loss: 1.4992 - accuracy: 0.6410
Epoch 91/200
1/1 [==============================] - 0s 16ms/step - loss: 1.4882 - accuracy: 0.6450
Epoch 92/200
1/1 [==============================] - 0s 15ms/step - loss: 1.4515 - accuracy: 0.6740
Epoch 93/200
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 0.1707 - accuracy: 0.9990
Epoch 166/200
1/1 [==============================] - 0s 13ms/step - loss: 0.1663 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 13ms/step - loss: 0.1610 - accuracy: 0.9990
Epoch 168/200
1/1 [==============================] - 0s 20ms/step - loss: 0.1568 - accuracy: 0.9990
Epoch 169/200
1/1 [==============================] - 0s 14ms/step - loss: 0.1523 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 12ms/step - loss: 0.1481 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 18ms/step - loss: 0.1441 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 15ms/step - loss: 0.1402 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 18ms/step - loss: 0.1363 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 14ms/step - loss: 0.1327 - accuracy: 1.0000
Epoch 175/200


In [11]:
query = 'some trees such as southern oaks maintain the [mask] width of ring for hundreds of years'

query2 = 'some trees such as southern oaks maintain the same width of ring for hundreds of years'

query_token_ids = tokenizer.texts_to_sequences([query])
query_token_ids = pad_sequences(query_token_ids, maxlen=MAX_SEQ_LEN, padding='post')

pred = model(query_token_ids)
pred_seq = np.ravel(pred.numpy().argmax(axis=-1))
pred_text = ' '.join(tokenizer.index_word[token] for token in pred_seq if token != 0)

print('actual text : ',query2)
print('----------------------------------------------------------------------')
print('masked text : ',query)
print('-----------------------------------------------------------------------')
print('predicted text : ',pred_text)

actual text :  some trees such as southern oaks maintain the same width of ring for hundreds of years
----------------------------------------------------------------------
masked text :  some trees such as southern oaks maintain the [mask] width of ring for hundreds of years
-----------------------------------------------------------------------
predicted text :  some trees such as southern oaks maintain the same of ring for hundreds of years


In [13]:
query = 'in the case of forest grown trees so much depends upon the competition of the trees in their struggle for [mask] and [mask] that periods of rapid and slow growth may alternate'

query2 = 'in the case of forest grown trees so much depends upon the competition of the trees in their struggle for light and nourishment that periods of rapid and slow growth may alternate'

query_token_ids = tokenizer.texts_to_sequences([query])
query_token_ids = pad_sequences(query_token_ids, maxlen=MAX_SEQ_LEN, padding='post')

pred = model(query_token_ids)
pred_seq = np.ravel(pred.numpy().argmax(axis=-1))
pred_text = ' '.join(tokenizer.index_word[token] for token in pred_seq if token != 0)

print('actual text : ',query2)
print('----------------------------------------------------------------------')
print('masked text : ',query)
print('-----------------------------------------------------------------------')
print('predicted text : ',pred_text)

actual text :  in the case of forest grown trees so much depends upon the competition of the trees in their struggle for light and nourishment that periods of rapid and slow growth may alternate
----------------------------------------------------------------------
masked text :  in the case of forest grown trees so much depends upon the competition of the trees in their struggle for [mask] and [mask] that periods of rapid and slow growth may alternate
-----------------------------------------------------------------------
predicted text :  in the case of forest grown trees so much depends upon the competition of the trees in their struggle light and that nourishment of periods and rapid growth slow may alternate
